In [201]:
from pyspark.sql import DataFrameReader
from pyspark.sql import SparkSession
from pyspark.ml.feature import IndexToString, Normalizer, StringIndexer, VectorAssembler, VectorIndexer
from pyspark.ml.classification import DecisionTreeClassifier
from helpers.helper_functions import translate_to_file_string
from pyspark.sql.functions import col,lit,to_date
from pyspark.sql.functions import expr,when
from pyspark.ml.feature import Imputer
from pyspark.ml.feature import StringIndexer
from sklearn.impute import KNNImputer
from pyspark.sql.functions import rand, desc
from pyspark.ml.feature import StringIndexer
from pyspark.ml import Pipeline
from pyspark.ml.feature import MinHashLSH
from pyspark.ml.feature import BucketedRandomProjectionLSH
from pyspark.sql import Row
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import MinMaxScaler
import random


# for pretty printing
def printDf(sprkDF): 
    newdf = sprkDF.toPandas()
    from IPython.display import display, HTML
    return HTML(newdf.to_html())

inputFile = translate_to_file_string("./data/RKI_COVID19_20210529.csv")

### Create Spark Session

In [2]:
#create a SparkSession
spark = (SparkSession
       .builder
       .appName("RKICOVID19PREPARATION")
       .getOrCreate())
# create a DataFrame using an ifered Schema 
df = spark.read.option("header", "true") \
       .option("inferSchema", "true") \
       .option("delimiter", ",") \
       .csv(inputFile)   
print(df.printSchema())

root
 |-- ObjectId: integer (nullable = true)
 |-- IdBundesland: integer (nullable = true)
 |-- Bundesland: string (nullable = true)
 |-- Landkreis: string (nullable = true)
 |-- Altersgruppe: string (nullable = true)
 |-- Geschlecht: string (nullable = true)
 |-- AnzahlFall: integer (nullable = true)
 |-- AnzahlTodesfall: integer (nullable = true)
 |-- Meldedatum: string (nullable = true)
 |-- IdLandkreis: integer (nullable = true)
 |-- Datenstand: string (nullable = true)
 |-- NeuerFall: integer (nullable = true)
 |-- NeuerTodesfall: integer (nullable = true)
 |-- Refdatum: string (nullable = true)
 |-- NeuGenesen: integer (nullable = true)
 |-- AnzahlGenesen: integer (nullable = true)
 |-- IstErkrankungsbeginn: integer (nullable = true)
 |-- Altersgruppe2: string (nullable = true)

None


In [3]:
df.count()

2003106

# Data Preperation

## Datenreinigung
Nimmt das Feld NeuerFall den Wert -1 an, so ist er laut RKI "[...] nur in der Publikation des Vortags enthalten". Das heißt es handelt sich folglich um eine Korrektur, der Puplikation des Vortages und muss in der aktuellen Datenauswertung näher betrachtet werden. Daraus ergibt sich, dass die entsprechenden Records aus dem Dataframe herausgefiltert werden müssen.

In [4]:
df = df.filter(df.NeuerFall > -1)

In [5]:
df.count()

2002544

## Datentransformation
In der Spalte AnzahlFall steht jeweils die Summe der Fälle. Um nun die verschiedenen Modelle trainieren zu können, muss nun die Aggration rückgängig gemacht werden. Das heißt für jeden Fall muss nun ein Record im DataFrame aufgenommen werden. Die neue Anzahl der Records muss der Anzahl der gemeldeten Fälle entsprechen. Darüber hinaus ist eine neue Spalte anzufügen. Die neue Spalte gibt an ob die Person genesen, gestorben oder keins von beiden ist. Die Spalten AnzahlFall, AnzahlTodesfall und AnzahlGenesen können dann entfallen. Ebenfalls die Felder NeuerFall, NeuerTodesfall und NeuGenesen. 

### Vereinzelung

In [6]:
df = df.withColumn("AnzahlFall", expr("explode(array_repeat(AnzahlFall,int(AnzahlFall)))"))
df.count()

3675296

### Neue Spalte

In [7]:
df = df.withColumn("FallStatus", when(df.AnzahlGenesen > 0, "GENESEN")
                                 .when(df.AnzahlTodesfall > 0, "GESTORBEN")
                                 .otherwise("NICHTEINGETRETEN"))

In [8]:
df.limit(10).show()

+--------+------------+------------------+------------+------------+----------+----------+---------------+--------------------+-----------+--------------------+---------+--------------+--------------------+----------+-------------+--------------------+-----------------+----------+
|ObjectId|IdBundesland|        Bundesland|   Landkreis|Altersgruppe|Geschlecht|AnzahlFall|AnzahlTodesfall|          Meldedatum|IdLandkreis|          Datenstand|NeuerFall|NeuerTodesfall|            Refdatum|NeuGenesen|AnzahlGenesen|IstErkrankungsbeginn|    Altersgruppe2|FallStatus|
+--------+------------+------------------+------------+------------+----------+----------+---------------+--------------------+-----------+--------------------+---------+--------------+--------------------+----------+-------------+--------------------+-----------------+----------+
|       1|           1|Schleswig-Holstein|SK Flensburg|     A15-A34|         M|         3|              0|2021/03/19 00:00:...|       1001|29.05.2021, 00:

## Datenreduktion
In dem folgenden Schritt werden die nicht notwendigen Spalten gelöscht. Spalte Altersgruppe2 ist nicht mit konkreten Werten befüllt und kann daher entfernt werden. Die Informationen aus AnzahlTodesfall und AnzahlGenesen bzw. NeuerTodefall und NeuGenesen sind durch das neue Feld FallStatus abgebildet. Die Felder AnzahlFall und NeuerFall sind durch die Vereinzelung und das Herausfiltern von Korrekturwerten überflüssig geworden. Die ObjectId hat an dieser Stelle auch keine Aussagekraft, da nicht mehrere Puplikationen verglichen werden, sonderns jeweils nur die Aktuelle Puplikation betrachtet wird. Ebenfalls nicht notwendig ist das Feld "IstErkrankungsbeginn". Das Feld Datenstand ist für alle Records das selbe Datum. Wird am heutigen Tag der RKI-Datensatz heruntergeladen enthält das Feld Datenstand das aktuelle Datum. Daher ist dieses nicht für die weitere Verarbeitung notwendig. Die Datenreduktion ist im Zuge der Feature Selection erfolgt. 

In [9]:
# definition der zu löschenden Spalten
columnsToDelete = ("Altersgruppe2", "AnzahlFall", "NeuerFall", "AnzahlTodesfall", "NeuerTodesfall", "AnzahlGenesen", "NeuGenesen", "IstErkrankungsbeginn", "Datenstand", "ObjectId")
df = df.drop(*columnsToDelete)

In [10]:
#Zeige die ersten Zehn Einträge
df.limit(10).show()

+------------+------------------+------------+------------+----------+--------------------+-----------+--------------------+----------+
|IdBundesland|        Bundesland|   Landkreis|Altersgruppe|Geschlecht|          Meldedatum|IdLandkreis|            Refdatum|FallStatus|
+------------+------------------+------------+------------+----------+--------------------+-----------+--------------------+----------+
|           1|Schleswig-Holstein|SK Flensburg|     A15-A34|         M|2021/03/19 00:00:...|       1001|2021/03/16 00:00:...|   GENESEN|
|           1|Schleswig-Holstein|SK Flensburg|     A15-A34|         M|2021/03/19 00:00:...|       1001|2021/03/16 00:00:...|   GENESEN|
|           1|Schleswig-Holstein|SK Flensburg|     A15-A34|         M|2021/03/19 00:00:...|       1001|2021/03/16 00:00:...|   GENESEN|
|           1|Schleswig-Holstein|SK Flensburg|     A15-A34|         M|2021/03/19 00:00:...|       1001|2021/03/19 00:00:...|   GENESEN|
|           1|Schleswig-Holstein|SK Flensburg|  

## Imputation fehlender Werte
Ein Teil der Datenreinigung ist die Impautation fehlder Werte. Fehlende Werte treten häufig in Datensätzen auf. Dies kann zu Problemen während dem Modelling führen. Aus diesem Grund gibt es verschiedene Möglichkeiten damit umzugehen. Das einfache Löschen von Datensätzen mit fehlenden Werten kann zu einer Verzerrung führen. Daher werden of Machine Learing Techniken angewandt um plausible Werte für die einzelen Features zu finden. (García et al. 2016, 4) In dem vorliegenden Datensatz des RKIs ist in bestimmten Fällen das Geschlecht bzw. das Alter unbekannt.
### Imputation des Geschlechts
Da der der Imputer von PySpark jedoch nicht für kategorische Werte geignet ist (Apache Spark 2021), wurde an dieser Stelle, das Verhältnis zwischen dem Männlichen sowie dem Weiblichen Geschlecht ermittelt und per Zufallswert das Geschlecht vergeben.

In [11]:
countWoman = df.filter(df.Geschlecht == "W").count()
countMan = df.filter(df.Geschlecht == "M").count()
countAll = countWoman + countMan
print("Anzahl Frauen: ", countWoman)
print("Anzahl Männer: ", countMan)
print("Gesamtzahl aller Datensätze mit vergebenem Geschlecht: ", countAll)

Anzahl Frauen:  1880476
Anzahl Männer:  1769786
Gesamtzahl aller Datensätze mit vergebenem Geschlecht:  3650262


In [12]:
df = df.withColumn("random", (rand() * countAll))
df = df.withColumn("randomGender", when(df.random > countWoman, "M").otherwise("W"))
df = df.withColumn("Geschlecht", when(df.Geschlecht == "unbekannt", df.randomGender).otherwise(df.Geschlecht)).drop("random","randomGender")

In [13]:
#Prüfung ob Datensatz nur korrekt:
df.filter(df.Geschlecht == "unbekannt").count()

0

## StringIndexer
Da viele Modelle nur mit numerischen Werten arbeiten können, müssen nicht numerische Features mittels eines StringIndexers in numerische Features umgewandelt werden. Dies erfolgt mithilfe des StringIndexers. Dies erfolgt für die Altersgruppe und das Geschlecht. Um die beiden Indexer miteinder zu verketten, wird an dieser Stelle eine Pipeline verwendet.

In [14]:
altersgruppeIndexer = StringIndexer(inputCol="Altersgruppe", outputCol="AltersgruppeIndex")
geschlechtsIndexer = StringIndexer(inputCol="Geschlecht", outputCol="GeschlechtIndex")
fallstatusIndexer = StringIndexer(inputCol="FallStatus", outputCol="FallStatusIndex")
pipeline = Pipeline(stages=[altersgruppeIndexer, geschlechtsIndexer,fallstatusIndexer])
df = pipeline.fit(df).transform(df)

In [15]:
#Zeige die ersten Zehn Einträge
df.limit(10).show()

+------------+------------------+------------+------------+----------+--------------------+-----------+--------------------+----------+-----------------+---------------+---------------+
|IdBundesland|        Bundesland|   Landkreis|Altersgruppe|Geschlecht|          Meldedatum|IdLandkreis|            Refdatum|FallStatus|AltersgruppeIndex|GeschlechtIndex|FallStatusIndex|
+------------+------------------+------------+------------+----------+--------------------+-----------+--------------------+----------+-----------------+---------------+---------------+
|           1|Schleswig-Holstein|SK Flensburg|     A15-A34|         M|2021/03/19 00:00:...|       1001|2021/03/16 00:00:...|   GENESEN|              1.0|            1.0|            0.0|
|           1|Schleswig-Holstein|SK Flensburg|     A15-A34|         M|2021/03/19 00:00:...|       1001|2021/03/16 00:00:...|   GENESEN|              1.0|            1.0|            0.0|
|           1|Schleswig-Holstein|SK Flensburg|     A15-A34|         M|

### Imputation der Altersgruppe
Für die Imputation von Kategoriewerten bietet PySpark kein StandardImputer. Mit viel Aufwand könnte über das Nearest-Neighbor Modell ein Imputer gebaut werden. Um an dieser Stelle darauf zu verzichten, wurde der KNNImputer der Sklearn-Lib verwendet. Dieser Schritt lässt sich jedoch dann nicht Pyspark-Cluster ausführen.

In [120]:
assembler =  VectorAssembler(outputCol="features", inputCols=["FallStatusIndex", "GeschlechtIndex","IdLandkreis"])
featureVector = assembler.transform(df)


In [121]:
scaler = MinMaxScaler(inputCol="features",outputCol="scaledFeatures")

scalerModel = scaler.fit(featureVector)
scaledFeatureVector= scalerModel.transform(featureVector)                                          


In [272]:
scaledFeatureVector = featureVector

In [126]:
#MinMaxScaler lässt nicht zu, dass Input = Outputcolumn ist. Daher wird an der Stelle nochmal die Feature-Column durch die ScaledFeatures ersetzt
scaledFeatureVector = scaledFeatureVector.withColumn("features", scaledFeatureVector.scaledFeatures).drop("scaledFeatures")

In [273]:
trainingFeatureVector = scaledFeatureVector.filter(df.Altersgruppe != "unbekannt");
targetFeatureVector = scaledFeatureVector.filter(df.Altersgruppe == "unbekannt");

In [128]:
trainingFeatureVector.groupBy("features").count().limit(10).show()

+--------------------+-----+
|            features|count|
+--------------------+-----+
|[0.0,0.0,0.003581...| 1679|
|[0.5,1.0,0.143075...|   43|
|[1.0,0.0,0.276001...|  139|
|[1.0,1.0,0.302003...|  146|
|[0.5,0.0,0.330127...|  201|
|[0.0,1.0,0.372114...| 3592|
|[0.0,1.0,0.418877...|  770|
|[0.5,0.0,0.491841...|  114|
|[0.5,1.0,0.542385...|   64|
|[1.0,1.0,0.542849...|   36|
+--------------------+-----+



In [130]:
trainingFeatureVector.limit(10).show()

+------------+------------------+------------+------------+----------+--------------------+-----------+--------------------+----------+-----------------+---------------+---------------+-------------+
|IdBundesland|        Bundesland|   Landkreis|Altersgruppe|Geschlecht|          Meldedatum|IdLandkreis|            Refdatum|FallStatus|AltersgruppeIndex|GeschlechtIndex|FallStatusIndex|     features|
+------------+------------------+------------+------------+----------+--------------------+-----------+--------------------+----------+-----------------+---------------+---------------+-------------+
|           1|Schleswig-Holstein|SK Flensburg|     A15-A34|         M|2021/03/19 00:00:...|       1001|2021/03/16 00:00:...|   GENESEN|              1.0|            1.0|            0.0|[0.0,1.0,0.0]|
|           1|Schleswig-Holstein|SK Flensburg|     A15-A34|         M|2021/03/19 00:00:...|       1001|2021/03/16 00:00:...|   GENESEN|              1.0|            1.0|            0.0|[0.0,1.0,0.0]|


In [131]:
trainingFeatureVector.count()

3671974

In [162]:
trainingFeatureVector.groupBy("features").count().count()

2472

In [274]:
trainingFeatureVectorGrouped = trainingFeatureVector.groupBy("features","Altersgruppe").count().orderBy(desc("features"))
trainingFeatureVectorGrouped.limit(10).show()

+-----------------+------------+-----+
|         features|Altersgruppe|count|
+-----------------+------------+-----+
|[2.0,1.0,16077.0]|     A35-A59|    6|
|[2.0,1.0,16077.0]|     A60-A79|   48|
|[2.0,1.0,16077.0]|        A80+|   86|
|[2.0,1.0,16077.0]|     A15-A34|    1|
|[2.0,1.0,16076.0]|     A35-A59|    3|
|[2.0,1.0,16076.0]|        A80+|   69|
|[2.0,1.0,16076.0]|     A60-A79|   42|
|[2.0,1.0,16075.0]|     A60-A79|   34|
|[2.0,1.0,16075.0]|        A80+|   55|
|[2.0,1.0,16075.0]|     A35-A59|    7|
+-----------------+------------+-----+



In [133]:
trainingFeatureVectorGrouped.count()

12119

In [275]:
#bucketRandomProjection = ,numHashTables=5
#MinHashLSH(inputCol="features", outputCol="hashes", numHashTables=5)
# mit NumHashTables=1 läuft die Erkennung pro Instanz innerhalb von 20 Sekunden
#MinHashLSH(inputCol="features", outputCol="hashes", numHashTables=1)


mhLSH = BucketedRandomProjectionLSH(inputCol="features", outputCol="hashes", bucketLength=2.0,numHashTables=1)
model = mhLSH.fit(trainingFeatureVectorGrouped)
transformedTrainingFeatureVector = model.transform(trainingFeatureVectorGrouped)

LSH OR-amplification can be used to reduce the false negative rate. Higher values for this param lead to a reduced false negative rate, at the expense of added computational complexity.
https://spark.apache.org/docs/latest/api/java/org/apache/spark/ml/feature/BucketedRandomProjectionLSH.html
The length of each hash bucket, a larger bucket lowers the false negative rate. The number of buckets will be (max L2 norm of input vectors) / bucketLength.

If input vectors are normalized, 1-10 times of pow(numRecords, -1/inputDim) would be a reasonable value

In [147]:
transformedTrainingFeatureVector.limit(10).show()

+--------------------+------------+-----+--------------------+
|            features|Altersgruppe|count|              hashes|
+--------------------+------------+-----+--------------------+
|[1.0,1.0,0.999999...|        A80+|   86|[[-1.0], [-1.0], ...|
|[1.0,1.0,0.999999...|     A15-A34|    1|[[-1.0], [-1.0], ...|
|[1.0,1.0,0.999999...|     A35-A59|    6|[[-1.0], [-1.0], ...|
|[1.0,1.0,0.999999...|     A60-A79|   48|[[-1.0], [-1.0], ...|
|[1.0,1.0,0.999933...|     A35-A59|    3|[[-1.0], [-1.0], ...|
|[1.0,1.0,0.999933...|     A60-A79|   42|[[-1.0], [-1.0], ...|
|[1.0,1.0,0.999933...|        A80+|   69|[[-1.0], [-1.0], ...|
|[1.0,1.0,0.999867...|     A35-A59|    7|[[-1.0], [-1.0], ...|
|[1.0,1.0,0.999867...|        A80+|   55|[[-1.0], [-1.0], ...|
|[1.0,1.0,0.999867...|     A60-A79|   34|[[-1.0], [-1.0], ...|
+--------------------+------------+-----+--------------------+



In [140]:
targetFeatureVector.groupBy("features").count().show(10, False)

+-------------------------------+-----+
|features                       |count|
+-------------------------------+-----+
|[0.0,0.0,0.0035818519501193947]|1    |
|[0.0,1.0,0.41887768638896256]  |3    |
|[0.5,0.0,0.5545900769434863]   |1    |
|[0.0,0.0,0.5744892544441496]   |2    |
|[0.0,0.0,0.4716105067657203]   |3    |
|[0.0,0.0,0.47850888829928356]  |1    |
|[0.0,1.0,0.6637702308304589]   |34   |
|[0.0,1.0,0.3293313876359777]   |1    |
|[0.0,0.0,0.47983550013266113]  |4    |
|[0.0,0.0,0.6634385778721146]   |10   |
+-------------------------------+-----+
only showing top 10 rows



In [136]:
targetFeatureVector.groupBy("features").count().count()

445

In [156]:
ag = model.approxNearestNeighbors(transformedTrainingFeatureVector, Vectors.dense([0.5,0.0,0.5545900769434863]), 6).first().Altersgruppe
print(ag)
#.orderBy(desc("count")).first().Altersgruppe
#groupBy("Altersgruppe").count().orderBy(desc("count")).first()

A05-A14


In [188]:
groupedRecords = model.approxNearestNeighbors(transformedTrainingFeatureVector, Vectors.dense([0.5,0.0,0.5545900769434863]), 6).orderBy(desc("count")).drop("features","hashes").collect()

In [189]:
print(groupedRecords)

[Row(Altersgruppe='A15-A34', count=16, distCol=0.0), Row(Altersgruppe='A35-A59', count=15, distCol=0.0), Row(Altersgruppe='A60-A79', count=11, distCol=0.0), Row(Altersgruppe='A05-A14', count=2, distCol=0.0), Row(Altersgruppe='A00-A04', count=2, distCol=0.0), Row(Altersgruppe='A80+', count=2, distCol=0.0)]


In [268]:
def getRandomAgeByFraction(recordList):
    sum = 0
    dictList = []
    for record in recordList :
        dict = record.asDict()
        print(dict)
        sum = sum + dict["count"]
        dictList.append(dict)

    percentSum = 0    
    for dict in dictList :
        dict["startPercent"] = percentSum
        dict["percent"] = dict["count"] / sum
        percentSum = percentSum + dict["percent"]
        dict["endPercent"] = percentSum


    randNr = random.random()
    Altergruppe = ""
    for dict in dictList :
        Altergruppe = dict["Altersgruppe"] if ((randNr >= dict["startPercent"]) & (randNr <= dict["endPercent"])) else Altergruppe
    
    return Altergruppe

print(getRandomAltersGroupByFraction(groupedRecords))

{'Altersgruppe': 'A15-A34', 'count': 16, 'distCol': 0.0}
{'Altersgruppe': 'A35-A59', 'count': 15, 'distCol': 0.0}
{'Altersgruppe': 'A60-A79', 'count': 11, 'distCol': 0.0}
{'Altersgruppe': 'A05-A14', 'count': 2, 'distCol': 0.0}
{'Altersgruppe': 'A00-A04', 'count': 2, 'distCol': 0.0}
{'Altersgruppe': 'A80+', 'count': 2, 'distCol': 0.0}
A15-A34


In [280]:
testdf = targetFeatureVector.limit(1)


resultList = []

for record in testdf.collect() :
    groupedRecords = model.approxNearestNeighbors(transformedTrainingFeatureVector, record.features, 6).orderBy(desc("count")).drop("features","hashes").collect()
    randAge = getRandomAgeByFraction(groupedRecords)    
    newRecord = Row(features=record.features, Altersgruppe=randAge)
    resultList.append(newRecord)
    
print("The Array is: ", resultList)

{'Altersgruppe': 'A15-A34', 'count': 507, 'distCol': 0.0}
{'Altersgruppe': 'A35-A59', 'count': 366, 'distCol': 0.0}
{'Altersgruppe': 'A05-A14', 'count': 92, 'distCol': 0.0}
{'Altersgruppe': 'A60-A79', 'count': 91, 'distCol': 0.0}
{'Altersgruppe': 'A00-A04', 'count': 46, 'distCol': 0.0}
{'Altersgruppe': 'A80+', 'count': 15, 'distCol': 0.0}
The Array is:  [Row(features=DenseVector([0.0, 1.0, 1001.0]), Altersgruppe='A05-A14')]


In [285]:
columns = ["features","AltersgruppeRandom"]
resultListDF = spark.createDataFrame(data=resultList, schema = columns)

In [286]:
resultListDF.limit(10).show()

+----------------+------------------+
|        features|AltersgruppeRandom|
+----------------+------------------+
|[0.0,1.0,1001.0]|           A05-A14|
+----------------+------------------+



In [300]:
joinedDF = featureVector.join(resultListDF, on='features', how='left')
joinedDF.show()

+-----------------+------------+----------+---------+------------+----------+--------------------+-----------+--------------------+----------+-----------------+---------------+---------------+------------------+
|         features|IdBundesland|Bundesland|Landkreis|Altersgruppe|Geschlecht|          Meldedatum|IdLandkreis|            Refdatum|FallStatus|AltersgruppeIndex|GeschlechtIndex|FallStatusIndex|AltersgruppeRandom|
+-----------------+------------+----------+---------+------------+----------+--------------------+-----------+--------------------+----------+-----------------+---------------+---------------+------------------+
|[0.0,0.0,16067.0]|          16| Thüringen| LK Gotha|     A00-A04|         W|2020/04/21 00:00:...|      16067|2020/04/21 00:00:...|   GENESEN|              5.0|            0.0|            0.0|              null|
|[0.0,0.0,16067.0]|          16| Thüringen| LK Gotha|     A00-A04|         W|2020/09/12 00:00:...|      16067|2020/09/12 00:00:...|   GENESEN|          

In [293]:
joinedDF.filter(joinedDF.AltersgruppeRandom.isNotNull() & (joinedDF.Altersgruppe == "unbekannt")).show()

+----------------+------------+------------------+------------+------------+----------+--------------------+-----------+--------------------+----------+-----------------+---------------+---------------+------------------+
|        features|IdBundesland|        Bundesland|   Landkreis|Altersgruppe|Geschlecht|          Meldedatum|IdLandkreis|            Refdatum|FallStatus|AltersgruppeIndex|GeschlechtIndex|FallStatusIndex|AltersgruppeRandom|
+----------------+------------+------------------+------------+------------+----------+--------------------+-----------+--------------------+----------+-----------------+---------------+---------------+------------------+
|[0.0,1.0,1001.0]|           1|Schleswig-Holstein|SK Flensburg|   unbekannt|         M|2021/01/09 00:00:...|       1001|2021/01/07 00:00:...|   GENESEN|              6.0|            1.0|            0.0|           A05-A14|
|[0.0,1.0,1001.0]|           1|Schleswig-Holstein|SK Flensburg|   unbekannt|         M|2021/01/09 00:00:...|    

In [337]:
dfAltersgruppeImputed = joinedDF.withColumn("Altersgruppe", when((joinedDF.Altersgruppe == "unbekannt") & (joinedDF.AltersgruppeRandom.isNotNull()), joinedDF.AltersgruppeRandom).otherwise(joinedDF.Altersgruppe)).drop("AltersgruppeRandom","features")

### AltersguppeIndex
Da die Altersgruppe veärndert wurde, muss hierfür nochmal der Index neu berechnet werden

In [338]:
altersgruppeIndexer = StringIndexer(inputCol="Altersgruppe", outputCol="AltersgruppeIndexNeu")
altersgruppeModel = altersgruppeIndexer.fit(dfAltersgruppeImputed)
dfWithAgeIndexNew = altersgruppeModel.transform(dfAltersgruppeImputed)
finalDF = dfWithAgeIndexNew.withColumn("AltersgruppeIndex", dfWithAgeIndexNew.AltersgruppeIndexNeu).drop("AltersgruppeIndexNeu")
finalDF.show()


+------------+----------+---------+------------+----------+--------------------+-----------+--------------------+----------+-----------------+---------------+---------------+
|IdBundesland|Bundesland|Landkreis|Altersgruppe|Geschlecht|          Meldedatum|IdLandkreis|            Refdatum|FallStatus|AltersgruppeIndex|GeschlechtIndex|FallStatusIndex|
+------------+----------+---------+------------+----------+--------------------+-----------+--------------------+----------+-----------------+---------------+---------------+
|          16| Thüringen| LK Gotha|     A00-A04|         W|2020/04/21 00:00:...|      16067|2020/04/21 00:00:...|   GENESEN|              5.0|            0.0|            0.0|
|          16| Thüringen| LK Gotha|     A00-A04|         W|2020/09/12 00:00:...|      16067|2020/09/12 00:00:...|   GENESEN|              5.0|            0.0|            0.0|
|          16| Thüringen| LK Gotha|     A00-A04|         W|2020/10/23 00:00:...|      16067|2020/10/23 00:00:...|   GENESEN| 

In [344]:
finalDF = finalDF.selectExpr("Bundesland", "IdBundesland as BundeslandIndex", "Landkreis", "IdLandkreis as LandkreisIndex", "Altersgruppe", "AltersgruppeIndex", "Geschlecht", "GeschlechtIndex", "FallStatus", "FallStatusIndex")

In [345]:
finalDF.repartition(1).write.format('csv').option('header',True).mode('overwrite').option('sep',';').save(translate_to_file_string("./data/data-preperation-result"))